In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
interventions = pd.read_csv('data/interventions.csv')

In [3]:
interventions.dropna(inplace=True)

In [4]:
line_out = [' et ', 'député', 'rapporteur', 'groupe', 'ministre', 'secrétaire', 'président', ',']
for element in line_out:
    interventions = interventions[interventions.nom.map(lambda x : element not in x)]

In [5]:
def calc_longueur(chaine):
    return len(chaine.split())
interventions['longueur'] = interventions.texte.map(calc_longueur)

In [6]:
mps = pd.read_csv('data/current_mps.csv', sep=';')

In [7]:
interventions_mps = interventions.merge(mps, on='nom', how='right')[['nom',
                                                                 'texte',
                                                                 'longueur']]
len(interventions_mps.nom.unique())

570

In [8]:
interventions_mps.texte = interventions_mps.texte.astype(str)
interventions_mps['clean_texte'] = interventions_mps['texte']

In [9]:
import string
ponctuations = string.punctuation + string.octdigits +'’«»'

In [10]:
for punct in ponctuations:
    interventions_mps['clean_texte'] = interventions_mps['clean_texte'].str.replace(punct, ' ')
interventions_mps['clean_texte'] = interventions_mps['clean_texte'].str.split()

/var/folders/32/9d66kzz52dz4b0_b57djs_nr0000gn/T/ipykernel_7911/192280174.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  interventions_mps['clean_texte'] = interventions_mps['clean_texte'].str.replace(punct, ' ')


In [11]:
def clean_liste(liste):
    resultat = []
    for elt in liste:
        if len(elt) > 2:
            resultat.append(elt)
    return resultat
def lowercase(liste):
    for index,word in enumerate(liste):
        liste[index] = word.strip().lower()
    return list(filter(None, liste))

In [12]:
interventions_mps['clean_texte'] = interventions_mps['clean_texte'].map(clean_liste)
interventions_mps['clean_texte'] = interventions_mps['clean_texte'].map(lowercase)

In [13]:
words_mps = {}
words_mps['nom'] = []
words_mps['mots'] = []
for nom in interventions_mps['nom'].unique():
    words_mp = interventions_mps[interventions_mps['nom'] == nom][['clean_texte']]
    words = []
    for texte in words_mp['clean_texte']:
        words += texte
    words_mps['nom'].append(nom)
    words_mps['mots'].append(words)
mots_mps = pd.DataFrame(words_mps)

In [14]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))
def clean_stopwords(liste):
    return [w for w in liste if not w in stop_words]

In [15]:
mots_mps['mots'] = mots_mps['mots'].map(clean_stopwords)

In [16]:
def get_number_of_occurences(liste):
    occurences = {}
    for s in liste:
        for w in s:
            if w not in occurences:
                occurences[w] = 0
            occurences[w] += 1
    return occurences
occurences = get_number_of_occurences(mots_mps['mots'])

In [17]:
nb_max = 50000
occ_sup = [k for k,v in occurences.items() if v >= nb_max]
print(len(occ_sup))
print(occ_sup)

0
[]


In [18]:
liste_a_oter = ['article', 'amendement', 'projet', 'loi', 'madame', 'monsieur', 'mesdames', 'messieurs', 'ministre', 'président', 'commission',
               'groupe', 'secrétaire', 'collègue', 'gouvernement', 'assemblée', 'rapporteur', 'chers', 'collègues', 'amendements', 'amendement',
               'majorité', 'défavorable', 'favorable', 'mme', 'avis', 'député', 'députés']

In [19]:
nb_min = 10
occ_inf = [k for k,v in occurences.items() if v <= nb_min]
print(len(occ_inf))
print(occ_inf)

49768
['hertziennes', 'télévisuels', 'cqp', 'valorisées', 'entrepreneuriale', 'microentreprises', 'bêtement', 'galderma', 'nestlé', 'sophia', 'antipolis', 'technopole', 'valetta', 'ardisson', 'naissant', 'nba', 'symbiose', 'unificateur', 'tolérées', 'plébiscite', 'concluantes', 'protagonistes', 'byod', 'incivilité', 'tapage', 'préférentiel', 'cyberhaine', 'cyberviolence', 'notifiant', 'matérialise', 'opci', 'preneurs', 'conciergerie', 'bayeux', 'lefèvre', 'financiariser', 'environnant', 'ravit', 'enchante', 'mouammar', 'kadhafi', 'exacerbation', 'préélémentaire', 'socioéconomique', 'esquissant', 'recrutant', 'turnover', 'niçoises', 'embouteillages', 'engageaient', 'débloquée', 'printaniers', 'estivaux', '…lors', 'télévisuelle', 'import', 'épilogue', 'manquerai', 'organisera', 'braillard', 'sympathisants', 'kaboul', 'sanctuarisées', 'nationalisés', 'vigie', 'visionnage', 'locomotive', 'supportérisme', 'coq', 'lemot', 'eim', 'libérons', 'bâtit', 'validations', 'fardeaux', 'porterai', 'we

In [20]:
def keep_words(X, occurences, max_occurence, min_occurence, liste):
    kept_words = {k:v for k, v in occurences.items() if v >= min_occurence and v <= max_occurence and k not in liste}
    output_sentences = [[word for word in sentence if word in kept_words] for sentence in X]    
    return output_sentences

In [21]:
mots_mps['clean_mots'] = keep_words(mots_mps['mots'], occurences, max_occurence=nb_max, min_occurence=nb_min, liste=liste_a_oter)

In [22]:
mots_mps['nbre_mots'] = mots_mps['clean_mots'].map(len)
mots_mps = mots_mps[['nom', 'clean_mots', 'nbre_mots']].sort_values(by='nbre_mots')

In [23]:
def most_frequent_words(liste):
    frequent_words = {}
    list_words = []
    for word in liste:
        if word not in list_words:
            frequent_words[word] = 0
            list_words.append(word)
        frequent_words[word] += 1    
    return frequent_words

In [24]:
mots_mps['clean_mots'] = mots_mps['clean_mots'].map(most_frequent_words)
mots_mps['nbre_mots'] = mots_mps['clean_mots'].map(len)

In [25]:
def list_frequent_words(dico):
    list_words = []
    for key in dico.keys():
        list_words.append(key)
    list_words.sort(key=lambda elt: dico[elt], reverse=True)
    return list_words

In [26]:
mots_mps['liste_mots'] = mots_mps['clean_mots'].map(list_frequent_words)

In [27]:
depute = interventions.groupby('nom')
interventions_mps = depute.count()
interventions_mps.drop(columns='longueur', inplace=True)
interventions_mps['longueur'] = depute['longueur'].mean()
interventions_mps

,texte,longueur
nom,,
Adrien Morenas,49,41.673469
Adrien Quatennens,1246,40.715891
Adrien Taquet,2091,39.454328
Agnès Buzyn,2890,42.942215
Agnès Firmin Le Bodo,739,49.637348
...,...,...
Éric Girardin,95,42.810526
Éric Pauget,490,47.936735
Éric Poulliat,236,45.953390


In [28]:
interventions_mps['nom'] = interventions_mps.index
interventions_mps.reset_index(inplace=True, drop=True)

In [29]:
def nettoyage(val):
    if np.isnan(val):
        val = 0
    return val
interventions_mps = interventions_mps.merge(mps, on='nom', how='inner')[['nom', 'texte', 'longueur', 'sexe', 'num_deptmt', 'parti_ratt_financier', 'id_an']]
interventions_mps.longueur = interventions_mps.longueur.map(nettoyage)
interventions_mps.sort_values(by='texte', inplace=True)
interventions_mps.reset_index(inplace=True, drop=True)

In [30]:
interventions_mps.rename(columns={'texte':'nbre_interv', 'longueur': 'mean_length'}, inplace=True)

In [31]:
interv_mps = interventions_mps.merge(mots_mps, on='nom', how='left')
interv_mps

,nom,nbre_interv,mean_length,sexe,num_deptmt,parti_ratt_financier,id_an,clean_mots,nbre_mots,liste_mots
0,Emmanuel Blairy,1,56.000000,H,62,NaN,720668,"{'permettez': 1, 'dire': 1, 'combien': 1, 'hon...",27,"[permettez, dire, combien, honoré, siéger, hém..."
1,Christophe Leclercq,2,30.000000,H,62,La République en Marche,784314,"{'plus': 1, 'enfants': 2, 'situation': 1, 'han...",28,"[enfants, handicap, plus, situation, scolarisé..."
2,Edith Audibert,2,94.000000,F,83,Les Républicains,721666,"{'question': 2, 'adresse': 1, 'état': 2, 'char...",91,"[tourisme, question, état, résidences, effet, ..."
3,Stéphane Vojetta,3,56.000000,H,999,La République en Marche,721170,"{'associe': 1, 'question': 1, 'représentant': ...",73,"[france, françaises, international, hors, comm..."
4,Bernard Brochand,3,4.000000,H,06,Les Républicains,223837,"{'actes': 1, 'entrées': 1, 'maîtrisé': 1}",3,"[actes, entrées, maîtrisé]"
...,...,...,...,...,...,...,...,...,...,...
561,Sébastien Jumel,5494,24.994175,H,76,Parti communiste français,722202,"{'premier': 81, 'ministres': 11, 'voici': 9, '...",9503,"[plus, bien, cette, cela, tout, être, fait, co..."
562,Fabien Di Filippo,6184,14.263745,H,57,Les Républicains,720386,"{'vrai': 97, 'comprends': 12, 'bien': 407, 'vo...",7485,"[plus, très, bien, tout, cela, fait, être, fau..."
563,Ugo Bernalicis,6549,33.668346,H,59,La France Insoumise,720430,"{'état': 388, 'veux': 59, 'tout': 814, 'abord'...",10968,"[plus, bien, tout, être, cela, fait, cette, fa..."
564,Pierre Cordier,6607,8.513092,H,08,Les Républicains,718850,"{'bravo': 61, 'très': 562, 'juste': 116, 'oui'...",5132,"[bien, très, oui, tout, fait, cela, question, ..."


In [32]:
interv_mps.to_pickle('data/mots_mps.pickle')

In [35]:
search_list = ['santé', 'solidarité']
def search_mps(dico):
    return all(item in dico.keys() for item in search_list)
search_mps = interv_mps[interv_mps.clean_mots.map(search_mps)][['nom', 'parti_ratt_financier', 'clean_mots', 'liste_mots']]
for keyword in search_list:
    def constr1(dico):
        return dico[keyword]
    def constr2(dico):
        nb_mots = sum([dico[key] for key in dico.keys()])
        return dico[keyword] * 100 / nb_mots
    search_mps[keyword] = search_mps['clean_mots'].map(constr1)
    search_mps[keyword + '_freq'] = search_mps['clean_mots'].map(constr2)
search_mps.drop(columns='clean_mots', inplace=True)
search_mps['total'] = 0
for column in search_list:
    search_mps['total'] += search_mps[column + '_freq']
search_mps.sort_values(by='total', inplace=True, ascending=False)
search_mps.drop(columns='total', inplace=True)
search_mps.head(10)

,nom,parti_ratt_financier,liste_mots,santé,santé_freq,solidarité,solidarité_freq
189,Charlotte Parmentier-Lecocq,La République en Marche,"[travail, santé, services, prévention, cette, ...",76,2.453988,1,0.032289
402,Stéphanie Rist,La République en Marche,"[santé, plus, donc, cette, cet, semble, étudia...",164,1.671082,1,0.010190
125,Marc Delatte,La République en Marche,"[santé, comme, cette, fait, éthique, aussi, to...",24,1.334816,6,0.333704
313,Jean-Carles Grelier,Les Républicains,"[santé, cette, plus, sans, comme, être, depuis...",87,1.440874,5,0.082809
300,Bernard Perrut,Les Républicains,"[santé, plus, bien, cette, tous, être, établis...",105,1.351004,6,0.077200
411,Cyrille Isaac-Sibille,Mouvement Démocrate,"[santé, plus, cette, bien, cet, tout, comme, ê...",150,1.347104,8,0.071846
281,Didier Martin,La République en Marche,"[santé, plus, cette, bien, conseil, cet, état,...",66,1.348866,2,0.040875
66,Michel Fanget,Mouvement Démocrate,"[santé, accord, cette, france, présidente, ent...",13,1.216090,1,0.093545
513,Thomas Mesnier,La République en Marche,"[santé, cet, déjà, plus, cette, donc, comme, t...",336,1.279074,8,0.030454
70,Sandrine Josso,"Union des démocrates, radicaux et libéraux","[plus, associations, santé, état, ainsi, depui...",10,0.759878,5,0.379939


In [36]:
parti = interv_mps.groupby('parti_ratt_financier')
interv_parti = parti.sum()
interv_parti['nbre_mps'] = parti['nom'].count()
interv_parti.drop(columns=['mean_length', 'nbre_mots', 'id_an'], inplace=True)
interv_parti['nbre_interv_mp'] = interv_parti['nbre_interv']/interv_parti['nbre_mps']
interv_parti['mean_length'] = parti['mean_length'].mean()
interv_parti['nbre_mots'] = parti['nbre_mots'].mean()
interv_parti

,nbre_interv,nbre_mps,nbre_interv_mp,mean_length,nbre_mots
parti_ratt_financier,,,,,
Calédonie Ensemble,470,2,235.000000,56.622431,2601.500000
Cap sur l'avenir,55,1,55.000000,57.472727,1006.000000
Debout la France,1158,3,386.000000,46.879725,2611.000000
Europe Écologie Les Verts,6149,12,512.416667,46.318794,3317.583333
La France Insoumise,39651,16,2478.187500,38.716940,7238.750000
La République en Marche,79156,271,292.088561,42.612154,1829.409594
Les Républicains,90464,102,886.901961,36.148378,2997.323529
Mouvement Démocrate,30348,68,446.294118,45.384598,2673.588235
Non déclaré(s),409,2,204.500000,37.154594,1617.000000


In [37]:
sexe = interv_mps.groupby('sexe')
interv_sexe = sexe.sum()
interv_sexe['nbre_mps'] = sexe['nom'].count()
interv_sexe.drop(columns=['mean_length', 'nbre_mots', 'id_an'], inplace=True)
interv_sexe['nbre_interv_mp'] = interv_sexe['nbre_interv']/interv_sexe['nbre_mps']
interv_sexe['mean_length'] = sexe['mean_length'].mean()
interv_sexe['nbre_mots'] = sexe['nbre_mots'].mean()
interv_sexe

,nbre_interv,nbre_mps,nbre_interv_mp,mean_length,nbre_mots
sexe,,,,,
F,95504,236,404.677966,45.256996,2346.987288
H,243892,330,739.066667,39.217916,2926.218182
